In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import gc
import PIL
import tensorflow as tf
from tensorflow import keras 


In [6]:
import pathlib
main_dir="../Dataset_l/PlantVillage/"
train_dir=os.path.join(main_dir,'train')
test_dir=os.path.join(main_dir,'test')
valid_dir=os.path.join(main_dir,'validation')

data_dir='../Dataset/PlantVillage'
#test_dir='Dataset//'
data_dir=pathlib.Path(data_dir)
#test_dir=pathlib.Path(test_dir)
data_dir


PosixPath('../Dataset/PlantVillage')

In [7]:
os.listdir(train_dir)

['2_Corn___common rust',
 '2_Corn___northern leaf blight',
 '2_Corn___cercospora leaf spot gray leaf spot',
 '1_Tomato___late blight',
 '0_Potato___late blight',
 '1_Tomato___target spot',
 '1_Tomato___leaf mold',
 '1_Tomato___healthy',
 '1_Tomato___bacterial spot',
 '1_Tomato___early blight',
 '2_Corn___healthy',
 '1_Tomato___septoria leaf spot',
 '1_Tomato___spider mites two-spotted spider mite',
 '1_Tomato___tomato yellow leaf curl virus',
 '0_Potato___early blight',
 '1_Tomato___tomato mosaic virus',
 '0_Potato___healthy']

In [8]:
gc.collect()


982

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    brightness_range=(0.3,0.9),
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

test_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(180,180),
    batch_size=64,
    class_mode='categorical'
)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(180,180),
    batch_size=64,
    class_mode='categorical'
)
valid_generator=train_datagen.flow_from_directory(
    valid_dir,
    target_size=(180,180),
    batch_size=64,
    class_mode='categorical'
) 

Found 15325 images belonging to 17 classes.
Found 2411 images belonging to 17 classes.
Found 4344 images belonging to 17 classes.


In [10]:
train_generator.class_indices

{'0_Potato___early blight': 0,
 '0_Potato___healthy': 1,
 '0_Potato___late blight': 2,
 '1_Tomato___bacterial spot': 3,
 '1_Tomato___early blight': 4,
 '1_Tomato___healthy': 5,
 '1_Tomato___late blight': 6,
 '1_Tomato___leaf mold': 7,
 '1_Tomato___septoria leaf spot': 8,
 '1_Tomato___spider mites two-spotted spider mite': 9,
 '1_Tomato___target spot': 10,
 '1_Tomato___tomato mosaic virus': 11,
 '1_Tomato___tomato yellow leaf curl virus': 12,
 '2_Corn___cercospora leaf spot gray leaf spot': 13,
 '2_Corn___common rust': 14,
 '2_Corn___healthy': 15,
 '2_Corn___northern leaf blight': 16}

In [ ]:
gc.collect()

In [ ]:
#plt.imshow(augmented_model(X)[0].numpy().astype("uint8"))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
num_classes=17
model=Sequential([
    layers.Conv2D(32,3, padding='same', input_shape=(180,180,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes,activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])



In [ ]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5,restore_best_weights=True,start_from_epoch=3)

In [ ]:
history=model.fit(train_generator,
          validation_data=valid_generator,
          epochs=20,
          batch_size=64,
          callbacks=[es]
         )

In [ ]:
model.save('model2.pb')

In [ ]:
plt.plot(history.epoch,history.history["accuracy"])
plt.plot(history.epoch,history.history["val_accuracy"])
plt.show()

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
import gc
gc.collect()

In [ ]:
model=keras.models.load_model('model2.pb')

In [ ]:
model.evaluate(test_generator)

In [ ]:
test_loss,test_acc=model.evaluate(test_generator)
print('test acc : {} test loss : {}'.format(test_acc,test_loss))

In [ ]:
#predictions=[np.argmax(model.predict(np.array([X_test_scaled[i]])))==y_test[i] for i in range(len(X_test_scaled))]